<a href="https://colab.research.google.com/github/AkhilSai13/Automated-Stock-Trading-Bot/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as y

In [ ]:
dataset = y.download('TCS.NS', start = '2010-01-01', end = '2014-12-31')
dataset2 = y.download('TCS.NS', start = '2015-01-01', end = '2020-01-01')
len(dataset)
#dataset.head() 


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10)) 
plt.plot(dataset['Close'])
plt.xlabel('Date')
plt.ylabel('Prices')
plt.title('TCS Closing Prices')

In [ ]:
import math
from sklearn.preprocessing import MinMaxScaler 
import numpy as np


close = dataset['Close']
close2 = dataset2['Close']
open = dataset2['Open']
arr_open = open.values
arr_close2 = close2.values
arr_close = close.values
arr = list(arr_close) + list(arr_close2)
arr = np.array(arr)
#print(len(arr))
#print(len(arr_close), len(arr_close2))
#print(arr_close[0],arr[0], arr[-1], arr_close2[-1])
scale = MinMaxScaler(feature_range=(0,1))
scaled = scale.fit_transform(arr_close.reshape(-1,1))
#scaled2 = scale.fit_transform(arr.reshape(-1,1))
#train_data_len = math.ceil(len(arr_close)*0.8)
#train_data_len = math.ceil(len(arr_close)-14)
train_data_len = len(arr_close)
train_data = scaled[0:train_data_len, :]   

x_train = []
y_train = []

for i in range(30, len(train_data)):
    x_train.append(train_data[i-30:i, 0])
    y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


In [ ]:
test_data = scaled[train_data_len-30: , : ]
x_test = []
#y_test = arr_close[train_data_len:]

#for i in range(14, len(test_data)):
x_test.append(test_data[-30: , 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
global_x_test = x_test


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()
model.add(layers.LSTM(100, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(layers.LSTM(100, return_sequences=False))
model.add(layers.Dense(25))
model.add(layers.Dense(1))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size= 1, epochs=10)

In [ ]:
from tensorflow.python.ops.gen_array_ops import prevent_gradient_eager_fallback
import pandas
from datetime import date, timedelta
#predictions = model.predict(x_test)
#predictions = scale.inverse_transform(predictions)
#rmse = np.sqrt(np.mean(predictions - y_test)**2)
#rmse
#drange = pandas.date_range(date(2015,1,1), date(2020,1,1)-timedelta(days=1),freq='d')
drange = dataset2.reset_index()['Date']
dates = []
dates2 = dict()

for i in range(1229):
  predictions = []
  for j in range(14):
    pred = model.predict(np.array(x_test[0][-30:], ndmin = 3))
    x_test = np.array(np.append(x_test[0], pred, axis=0), ndmin = 3)
    print(scale.inverse_transform(pred))
    predictions.append(scale.inverse_transform(pred)[0][0])
  
  d = str(drange[i].strftime('%Y-%m-%d'))
  if(predictions[-1] >= arr_open[i]):
    dates.append(d)
    dates2[d] = 1
  else:
    #dates.append(0)
    dates2[d] = 0

  # train the model with new data
  arr_close = np.append(arr_close, [arr[1232 + i]], axis = 0)
  scaled = scale.fit_transform(arr_close.reshape(-1,1))
  #train_data = np.append(train_data, scaled, axis = 0)
  y_train = scaled[1232 + i]
  x_train = scaled[1232 + i -30 : 1232 + i]
  x_train, y_train = np.array(x_train, ndmin = 3), np.array(y_train)
  #x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1)) 
  model.fit(x_train, y_train, batch_size= 1, epochs=10)

  # add new data to global_x_test
  new_data = np.array(y_train, ndmin = 2)
  global_x_test = np.array(np.append(global_x_test[0], new_data, axis=0), ndmin = 3)

  x_test = global_x_test


'''
predictions = []
for i in range(14):
  #np.concatenate(predictions, model.predict(predictions[-14: ]))
  pred = model.predict(np.array(x_test[0][-30:], ndmin = 3))
  x_test = np.array(np.append(x_test[0], pred, axis=0), ndmin = 3)
  print(scale.inverse_transform(pred))
  predictions.append(scale.inverse_transform(pred)[0][0])
  
if(predictions[-1] <= arr_close2[0]):
  dates.append(0)
'''


In [ ]:
print(len(dates2))
print(len(dates))
print(predictions[0])
print(arr_close2[-1])